In [29]:
from bs4 import BeautifulSoup as bs
import requests as rq
import pandas as pd
import numpy as np
import re
from time import sleep
from time import time
from datetime import datetime
from dateutil.relativedelta import relativedelta
import json
import urllib
import random
from time import sleep

In [118]:
today_str = datetime.now().strftime("%Y-%m-%d")

In [119]:
three_months_ago_str = (datetime.now() - relativedelta(days=+90)).strftime("%Y-%m-%d")

In [ ]:
#TODO
# SCRAPE all coins from page 1 - CHECK
# SCRAPE EACH page up to page 10 and repeat previous step - CHECK
# convert and append all into one pandas dataframe - CHECK
# begin analysis:
# * calculate rank for each coin, per day - CHECK
# * find a way to detect positive trends, "upcoming stars"
# consider pushing to sql database again

In [21]:
pd.set_option('display.max_rows', 500)

In [120]:
def random_sleep():
    sleep(random.uniform(1, 3))

In [ ]:
df = pd.DataFrame(columns=['date', 'coin_name', 'symbol', 'market_cap', 'volume', 'price', '24_hr_pct_change', '7_day_pct_change', 'v_marketcap'])
start_time = time()
for i in range(1, 11):
    url = f'https://www.coingecko.com/en?page={i}'
    main_html = rq.get(url).text
    main_html_soup = bs(main_html,'html.parser')
    hrefs = main_html_soup.find_all("a", {"class": "d-lg-none font-bold"}, href=True)
    for coin_url in hrefs:
        try:
            df = df.append(scrape_coin(coin_url['href']))
            random_sleep() # sleep 1-3 seconds to avoid getting blocked by captcha
        except Exception as e:
            print(coin_url)
            print(e)
end_time = time()
print(f'Seconds passed: {end_time - start_time}')

In [121]:
def day_pct_change(x):
    return (x.iloc[1] - x.iloc[0]) / x.iloc[1] * 100

In [122]:
def seven_day_pct_change(x):
    return (x.iloc[6] - x.iloc[0]) / x.iloc[6] * 100

In [123]:
# Volume divided by marketcap
def calculate_v_marketcap(x):
    try:
        return x['volume'] / x['market_cap']
    except:
        return 0

In [124]:
def scrape_coin(coin_url):
    url = f'https://www.coingecko.com/{coin_url}/historical_data/usd?end_date={today_str}&start_date={three_months_ago_str}#panel'
    text = rq.get(url).text
    soup = bs(text,'html.parser')

    dates = [x.text.strip() for x in soup.find_all("th", {"class" : "font-semibold text-center"})]
    # Remove $ and commas from coin information (Market Cap, Volume, Price), to convert to int later
    info = [x.text.strip()[1:].replace(',', '') for x in soup.find_all("td", {"class" : "text-center"})]
    market_caps = info[0::4]
    volumes = info[1::4]
    open_price = info[2::4]
    # arr_length is calculated to fit coin symbol/name to dataframe. May differ from 60 if the coin is new.
    arr_length = len(market_caps) 
    coin = [x.text.strip() for x in soup.find_all("div", {"class" : "mr-md-3 mx-2 mb-md-0 text-3xl font-semibold"})]
    # example for coin: Bitcoin (BTC)
    coin_symbol = [re.findall('.*\((.*)\)', x)[0] for x in coin] * arr_length
    coin_name = [re.findall('(.*)[ ]\(', x)[0] for x in coin] * arr_length

    df = pd.DataFrame({'date' : dates, 'coin_name' : coin_name, 'symbol' : coin_symbol, 'market_cap' : market_caps, 'volume' : volumes, 'price' : open_price})
    df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%d")
    df = df.astype({'market_cap' : 'float64', 'volume' : 'float32', 'price' : 'float32'})
    # Pandas Rolling function is backwards; reversing market_cap series and then reversing the series returned will
    # implement a forward rolling function
    df['24_hr_pct_change'] = df['market_cap'][::-1].rolling(window=2).apply(lambda x: day_pct_change(x))[::-1]
    df['7_day_pct_change'] = df['market_cap'][::-1].rolling(window=7).apply(lambda x: seven_day_pct_change(x))[::-1]
    df['v_marketcap'] = df.apply(lambda x: calculate_v_marketcap(x), axis=1)
    df = df.astype({'24_hr_pct_change' : 'float16', '7_day_pct_change' : 'float16', 'v_marketcap' : 'float16'})
    
    return df

In [125]:
def day_rank_change(x):
    return x.iloc[1] - x.iloc[0]

In [126]:
def calculate_daily_rank_change(df):
    # Calculate daily rank change, per coin to test correlation between rank change and price
    temp_df = pd.DataFrame(columns = ['date', 'coin_name', 'symbol', 'market_cap', 'volume', 'price',
        '24_hr_pct_change', '7_day_pct_change', 'v_marketcap', 'rank', 'daily_rank_change'])
    for coin in df['coin_name'].unique():
        temp = df.loc[df['coin_name'] == coin]
        temp['daily_rank_change'] = temp['rank'][::-1].rolling(window=2).apply(lambda x: day_rank_change(x))[::-1]
        temp_df = temp_df.append(temp)
    return temp_df

In [127]:
# Calculate the ranking of the coins per day; ranking is determined by market cap
# To calculate daily rank for the last three months, the coins are sorted by market cap per day
def calculate_daily_rank(df):
    temp_df = pd.DataFrame(columns = ['date', 'coin_name', 'symbol', 'market_cap', 'volume', 'price',
        '24_hr_pct_change', '7_day_pct_change', 'v_marketcap', 'rank'])
    for date in df['date'].unique():
        date = pd.to_datetime(date).date()
        temp = df.loc[df['date'].dt.date == date]
        # Produces an array [1..{amount of rows}]
        temp['rank'] = np.arange((temp.sort_values('market_cap').shape[0]))+1
        temp_df = temp_df.append(temp)
    return temp_df

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57558 entries, 0 to 57557
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        57558 non-null  int64  
 1   date              57558 non-null  object 
 2   coin_name         57558 non-null  object 
 3   symbol            57558 non-null  object 
 4   market_cap        57558 non-null  float64
 5   volume            57558 non-null  float64
 6   price             57558 non-null  float64
 7   24_hr_pct_change  54381 non-null  float64
 8   7_day_pct_change  49962 non-null  float64
 9   v_marketcap       57558 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 4.2+ MB


In [112]:
df = df.drop_duplicates()
df = calculate_daily_rank(df)
df = calculate_daily_rank_change(df)

<ipython-input-102-99737a7294ad>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['rank'] = np.arange((temp.sort_values('market_cap').shape[0]))+1
<ipython-input-111-80652e592f98>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['daily_rank_change'] = temp['rank'][::-1].rolling(window=2).apply(lambda x: day_rank_change(x))[::-1]


In [ ]:
df = df.drop_duplicates()
df = calculate_daily_rank(df)
df = calculate_daily_rank_change(df)
df.to_csv('data.csv')

In [57]:
df = pd.read_csv('data.csv')
df['date'] = pd.to_datetime(df['date'])

In [28]:
df[(df['24_hr_pct_change'] > 15) & (df['7_day_pct_change'] < 0) & (df['rank'] > 250) & (df['rank'] < 300)]

,Unnamed: 0,date,coin_name,symbol,market_cap,volume,price,24_hr_pct_change,7_day_pct_change,v_marketcap,rank
276,0,2021-04-09,ThunderCore,TT,176364614.0,66321280.0,0.026182,16.75,-43.000,0.376000,277
13039,13,2021-03-27,LUKSO Token,LYXE,270438007.0,7640275.0,18.100000,23.61,-12.875,0.028240,251
13045,13,2021-03-27,IRISnet,IRIS,211531631.0,28175994.0,0.217686,20.06,-9.260,0.133200,257
13049,13,2021-03-27,iExec RLC,RLC,149230652.0,18558940.0,2.110000,15.21,-5.370,0.124400,261
18836,19,2021-03-21,LUKSO Token,LYXE,305261203.0,10561028.0,20.180000,18.47,-21.770,0.034600,251
30266,31,2021-03-09,CoinMetro,XCM,149867677.0,462669.0,0.494210,16.38,-11.320,0.003088,270
36803,38,2021-03-02,Atari,ATRI,59804437.0,143860.0,0.149596,15.30,-3.791,0.002405,262
36823,38,2021-03-02,Marlin,POND,52151008.0,12047979.0,0.119815,15.71,-4.746,0.231100,282
36834,38,2021-03-02,Beefy.Finance,BIFI,86414187.0,3434124.0,1126.620000,23.70,-4.230,0.039730,293
38664,40,2021-02-28,UnFederalReserve,ERSDL,19745964.0,3879329.0,0.072087,15.40,-7.434,0.196400,275
